<a href="https://colab.research.google.com/github/bdhavn6codemaster/AI-Website-Remixer2/blob/main/Remixer_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from PIL import Image
from IPython.display import display

# Streaming mode keeps your A14 from crashing
dataset = load_dataset("HuggingFaceM4/WebSight", split="train", streaming=True)
example = next(iter(dataset))

# DEBUG: This line shows you all available "labels" in the data
print(f"Available keys: {example.keys()}")

print("\n--- THE REMIXABLE CODE ---")
# Use 'text' instead of 'html'
print(example['text'][:1000])

print("\n--- THE ORIGINAL DESIGN ---")
img = example['image']
img.thumbnail((400, 400)) # Mobile-friendly size
display(img)
# THE REMIX LOGIC
original_code = example['text']

# Let's do a "Dark Mode" Remix
# We swap 'bg-white' for 'bg-slate-900' and 'text-black' for 'text-white'
remixed_code = original_code.replace("bg-white", "bg-slate-900").replace("text-black", "text-emerald-400")

print("--- YOUR REMIXED CODE (Dark Mode & Emerald Text) ---")
print(remixed_code[:1000])
from google.colab import files

# 1. Take your remixed code from the last step
# (Make sure the 'remixed_code' variable exists from our previous chat!)
final_html = remixed_code

# 2. Write it to a file on the Google Server
with open("remix.html", "w") as f:
    f.write(final_html)

    # 3. Send it to your phone
    files.download("remix.html")
    print("Check your phone's 'Downloads' folder!")

In [ ]:
# 1. Install the Groq library
!pip install -q groq datasets

import os
from groq import Groq
from datasets import load_dataset
from google.colab import userdata

# 2. Setup the "Brain"
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

# 3. Load a real website from our dataset
dataset = load_dataset("HuggingFaceM4/WebSight", split="train", streaming=True)
website = next(iter(dataset))
original_html = website['text'][:2000] # Just the first bit so it's fast

# 4. Ask the AI to Remix it
print("🧠 The AI is re-imagining the design...")

chat_completion = client.chat.completions.create(
    messages=[
            {
                        "role": "system",
                                    "content": "You are a world-class web designer. Your job is to take raw HTML and 'Remix' it. Keep the same buttons and links, but completely change the CSS styles to look like a 'Cyberpunk' theme with neon glows."
                                            },
                                                    {
                                                                "role": "user",
                                                                            "content": f"Here is the website code to remix: {original_html}"
                                                                                    }
                                                                                        ],
                                                                                            model="llama-3.3-70b-versatile",
                                                                                            )

                                                                                            # 5. Result
                                                                                            remixed_html = chat_completion.choices[0].message.content
                                                                                            print("\n--- YOUR AI-GENERATED REMIX ---")
                                                                                            print(remixed_html[:1000])


In [ ]:
import zipfile
from google.colab import files

# 1. Create a list to store our new websites
remixed_pages = []

# 2. Start the Loop: Grab 5 websites
# We use 'enumerate' to give each one a number (0, 1, 2, 3, 4)
for i, example in enumerate(dataset.take(5)):
    original_html = example['text']

    # THE REMIX: Change colors and the Title
    remixed = original_html.replace("bg-gray-100", "bg-black") \
                           .replace("text-gray-800", "text-green-400") \
                           .replace("Fashion Brand", f"AI Remix Store #{i}")

    filename = f"remix_{i}.html"

    # Save each one temporarily in Colab
    with open(filename, "w") as f:
        f.write(remixed)
    remixed_pages.append(filename)

# 3. ZIP THEM UP: One file is easier for your A14 to download
with zipfile.ZipFile('all_remixes.zip', 'w') as zipf:
    for file in remixed_pages:
        zipf.write(file)

# 4. DOWNLOAD TO PHONE
files.download('all_remixes.zip')
print("Your ZIP of 5 remixed sites is headed to your A14!")

In [ ]:
# 1. Install the Groq library
!pip install -q groq datasets

import os
from groq import Groq
from datasets import load_dataset
from google.colab import userdata

# 2. Setup the "Brain"
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

# 3. Load a real website from our dataset
dataset = load_dataset("HuggingFaceM4/WebSight", split="train", streaming=True)
website = next(iter(dataset))
original_html = website['text'][:2000] # Just the first bit so it's fast

# 4. Ask the AI to Remix it
print("🧠 The AI is re-imagining the design...")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a world-class web designer. Your job is to take raw HTML and 'Remix' it. Keep the same buttons and links, but completely change the CSS styles to look like a 'Cyberpunk' theme with neon glows."
        },
        {
            "role": "user",
            "content": f"Here is the website code to remix: {original_html}"
        }
    ],
    model="llama-3.3-70b-versatile",
)

# 5. Result
remixed_html = chat_completion.choices[0].message.content
print("\n--- YOUR AI-GENERATED REMIX ---")
print(remixed_html[:1000])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# 1. Create a Text Box for your 'Vibe'
vibe_input = widgets.Text(
    value='Cyberpunk neon style',
    placeholder='Type a theme (e.g. 1920s Jazz, Minimalist, Space Station)',
    description='Theme:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)

# 2. Create a Button to Trigger the Remix
run_button = widgets.Button(
    description='Generate Remix',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='90%', height='50px')
)

# 3. Create an Output Area to see the result
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"🚀 Remixing with theme: {vibe_input.value}...")

        # This calls the Groq AI logic we set up earlier
        try:
            # We reuse the 'original_html' from your previous cell
            prompt = f"Remix this HTML code into a {vibe_input.value} theme. Return ONLY the code."

            chat_completion = client.chat.completions.create(
                messages=[{"role": "user", "content": f"{prompt}\n\nCode: {original_html}"}],
                model="llama-3.3-70b-versatile",
            )

            remixed_result = chat_completion.choices[0].message.content

            # Display the result as a clickable preview link
            with open("mobile_remix.html", "w") as f:
                f.write(remixed_result)

            print("✅ Done! File saved as 'mobile_remix.html'")
            display(HTML(f"<b>Preview (Top 500 chars):</b><br><pre>{remixed_result[:500]}</pre>"))

        except Exception as e:
            print(f"❌ Error: {e}")

run_button.on_click(on_button_clicked)

# Display everything
display(vibe_input, run_button, output)

In [ ]:
def ultimate_remix(url, style_vibe):
    print(f"🌐 Scraping {url}...")

    # 1. Scrape the site
    scraped_data = firecrawl.scrape(url, formats=['html', 'screenshot'])

    # 2. Access properties using dot notation (v2 way)
    raw_html = scraped_data.html[:4000]
    metadata = scraped_data.metadata

    # NEW: Access metadata attributes directly
    # We use 'getattr' as a safety net in case the site doesn't have a title
    site_title = getattr(metadata, 'title', 'a website')
    site_desc = getattr(metadata, 'description', 'no description provided')

    print(f"🧠 AI is re-imagining '{site_title}'...")

    # 3. The Brain (Groq)
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{
            "role": "system",
            "content": f"""You are a master designer.
            CONTEXT: This website is '{site_title}' ({site_desc}).
            TASK: Remix the HTML into a {style_vibe} theme.
            Use only Tailwind CSS. Return ONLY the code."""
        },
        {"role": "user", "content": raw_html}]
    )

    # 4. Save and Display
    with open("live_remix.html", "w") as f:
        f.write(response.choices[0].message.content)

    if scraped_data.screenshot:
        print("📸 'Before' screenshot:")
        display(HTML(f'<img src="{scraped_data.screenshot}" width="400">'))

    print("✅ Done! File saved as 'live_remix.html'.")

# START THE REMIXER
ultimate_remix("https://example.com", "Futuristic Space Station")